<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab 3.02: Statistical Modeling and Model Validation

> Authors: Tim Book, Matt Brems

---

## Objective
The goal of this lab is to guide you through the modeling workflow to produce the best model you can. In this lesson, you will follow all best practices when slicing your data and validating your model. 

## Imports

In [2]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.linear_model import LinearRegression
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')


## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [3]:
# Read in the citibike data in the data folder in this repository.
ctb = pd.read_csv('./data/citibike_feb2014.csv')

## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

In [4]:
ctb.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,-73.989780,15456,Subscriber,1979,2
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,-73.994800,16281,Subscriber,1948,2
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,17400,Subscriber,1981,1
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,-73.989780,19341,Subscriber,1990,1


In [5]:
ctb.isnull().sum() # no null values

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
dtype: int64

In [6]:
ctb.dtypes 
# starttime and stoptime can be changed to datetime format
# birth year can be changed to integer format

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                  object
gender                       int64
dtype: object

In [7]:
ctb['starttime'] = pd.to_datetime(ctb['starttime'])
ctb['stoptime'] = pd.to_datetime(ctb['stoptime'])
# cast 'datetime' to actually be date time format

In [8]:
ctb['birth year'].unique() # there is a "birth year" that is called "\\N"

array(['1991', '1979', '1948', '1981', '1990', '1978', '1944', '1983',
       '1969', '1986', '1962', '1965', '1942', '1989', '1980', '1957',
       '1951', '1992', '1971', '1982', '1968', '1984', '\\N', '1956',
       '1987', '1985', '1996', '1975', '1988', '1974', '1972', '1959',
       '1973', '1977', '1976', '1953', '1993', '1970', '1963', '1967',
       '1966', '1960', '1961', '1994', '1958', '1955', '1946', '1964',
       '1900', '1995', '1954', '1952', '1949', '1947', '1941', '1938',
       '1950', '1945', '1997', '1934', '1940', '1939', '1936', '1943',
       '1935', '1937', '1922', '1932', '1907', '1926', '1899', '1901',
       '1917', '1910', '1933', '1921', '1927', '1913'], dtype=object)

In [9]:
ctb['birth year'] = ctb['birth year'].replace('\\N', '0') # replace \\N with 0
ctb['birth year'] = ctb['birth year'].astype(int) # change to integer format


## Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

$ H_0$: The true mean difference in trip duration between gender=1 and gender=2 is 0.

$ H_A$: The true mean difference in trip duration between gender=1 and gender=2 is NOT 0.

In [10]:
ctb_1 = ctb[ctb['gender'] == 1]
ctb_2 = ctb[ctb['gender'] == 2]
t_stat, p_value = stats.ttest_ind(ctb_1['tripduration'], ctb_2['tripduration'], equal_var=False)
p_value

1.5680482053980366e-06

Since the p-value is less than 0.05, we reject $ H_0$ and state $ H_A$. The true mean difference in trip duration between gender=1 and gender=2 is NOT 0.

## What numeric columns shouldn't be treated as numeric?

In [11]:
ctb['gender'].unique()

array([1, 2, 0], dtype=int64)

**Answer:** Gender. In the dataset, there are three genders: 0, 1 and 2. It would be better to use strings 'Male' and 'Female' or using dummy columns. 'bikeid' should be a string, because doing any operations on the 'bikeid' does not make sense.

## Dummify the `start station id` Variable

In [12]:
ctb = pd.get_dummies(columns=["start station id"], data=ctb, dtype=int, drop_first=True)

## Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected).

- Note: you will need to clean the data a bit.

In [13]:
ctb['age'] = 2014 - ctb['birth year']

## Split your data into train/test data

Look at the size of your data. What is a good proportion for your split? **Justify your answer.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.)

**NOTE:** When doing your train/test split, please use random seed 123.

In [14]:
y = ctb['tripduration'] # setting tripduration as y
X = ctb.drop(columns=['tripduration', 'starttime', 'stoptime', 'start station name', 'start station latitude','start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude', 'bikeid', 'birth year'])
# drop other columns that are not required
X = pd.get_dummies(columns=["usertype"], data=X, dtype=int) # creating dummy columns for subscriber and customers
X = X[['age'] + ['usertype_Customer'] + ['usertype_Subscriber'] + [col for col in X.columns if (col != 'age') and (col != 'usertype_Subscriber') and (col != 'usertype_Customer')]]
X

,age,usertype_Customer,usertype_Subscriber,gender,start station id_79,start station id_82,start station id_83,start station id_116,start station id_119,start station id_120,...,start station id_2006,start station id_2008,start station id_2009,start station id_2010,start station id_2012,start station id_2017,start station id_2021,start station id_2022,start station id_2023,start station id_3002
0,23,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,33,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224731,38,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224732,29,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224733,46,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224734,32,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
# we use the default proportion which is 75% train and 25% test
# the 25% test still has alot of data

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [16]:
model = LinearRegression()
# Fit model
model.fit(X_train, y_train)
# set predictions for X_test
preds = model.predict(X_test)

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

In [17]:
# Train score
model.score(X_train, y_train)

0.0050145707569594355

In [18]:
# Test score
model.score(X_test, y_test)

-0.0018590472125472601

In [19]:
print(f"MSE for train values is {metrics.mean_squared_error(y_train, model.predict(X_train))}")
print(f"MSE for test values is {metrics.mean_squared_error(y_test, preds)}")

MSE for train values is 30046595.00436887
MSE for test values is 29849542.668169316


In [20]:
print(f"R-squared for train values is {metrics.r2_score(y_train, model.predict(X_train))}")
print(f"R-squared for test values is {metrics.r2_score(y_test, preds)}")
# same values as model.score above

R-squared for train values is 0.0050145707569594355
R-squared for test values is -0.0018590472125472601


Since the R-squared values for both the train and test are very small, this is an underfit. 0.243% of the variability in trip duration can be explained by the other columns in X.

In [21]:
# y_hat is the mean of y_train values
y_hat = [np.mean(y_train)] * len(y_test)

In [22]:
print(f"MSE for baseline is {metrics.mean_squared_error(y_test, y_hat)}")

MSE for baseline is 29794242.345045418


In [23]:
print(f"R-squared for baseline is {metrics.r2_score(y_test, y_hat)}")

R-squared for baseline is -2.967551583799022e-06


Since the R-squared values of the train and test values are much larger than that of the baseline, this model outperforms the baseline.

## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [24]:
import statsmodels.api as sm
X_train = sm.add_constant(X_train)

In [26]:
model = sm.OLS(y_train, X_train).fit()

## Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

In [27]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           tripduration   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.561
Date:                Thu, 07 Mar 2024   Prob (F-statistic):           7.04e-47
Time:                        21:19:44   Log-Likelihood:            -1.6903e+06
No. Observations:              168552   AIC:                         3.381e+06
Df Residuals:                  168220   BIC:                         3.384e+06
Df Model:                         331                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2683.2643    843.835     -3.180      0.001   -4337.162   -1029.367
age                       5.2368      1.208      4.333      0.000       2.868       7.605
usertype_Customer     -5973.8486   1609.489     -3.712      0.000   -9128.412   -2819.285
usertype_Subscriber    3290.5842    784.419      4.195      0.000    1753.141    4828.028
gender                  193.9965     34.944      5.552      0.000     125.507     262.486
start station id_79      85.3919    351.187      0.243      0.808    -602.926     773.710
start station id_82     425.4700    448.380      0.949      0.343    -453.344    1304.284
start station id_83    -139.7571    434.738     -0.321      0.748    -991.834     712.319
start station id_116   -351.2927    300.904     -1.167      0.243    -941.058     238.472
start station id_119   -408.4159    845.508     -0.483      0.629   -2065.594    1248.762
start station id_120   1046.5993    709.298      1.476      0.140    -343.610    2436.808
start station id_127   -324.2846    317.340     -1.022      0.307    -946.265     297.695
start station id_128   -339.4219    307.303     -1.105      0.269    -941.728     262.884
start station id_137   -297.5796    376.439     -0.791      0.429   -1035.391     440.232
start station id_143   -347.2945    524.771     -0.662      0.508   -1375.834     681.245
start station id_144   -113.8066    687.586     -0.166      0.869   -1461.459    1233.846
start station id_146   -430.9635    368.504     -1.169      0.242   -1153.224     291.297
start station id_147    -82.6440    321.681     -0.257      0.797    -713.131     547.843
start station id_150   -389.3339    365.780     -1.064      0.287   -1106.254     327.586
start station id_151   -354.7140    287.389     -1.234      0.217    -917.990     208.562
start station id_152   -405.8956    405.157     -1.002      0.316   -1199.994     388.203
start station id_153   -270.9595    314.236     -0.862      0.389    -886.855     344.936
start station id_157     86.0323    428.639      0.201      0.841    -754.092     926.156
start station id_160   -230.8814    350.433     -0.659      0.510    -917.723     455.960
start station id_161   -396.5275    305.438     -1.298      0.194    -995.179     202.124
start station id_164   -345.3254    350.623     -0.985      0.325   -1032.540     341.889
start station id_167   -230.5140    318.459     -0.724      0.469    -854.686     393.658
start station id_168   -287.1817    299.074     -0.960      0.337    -873.361     298.997
start station id_173   -401.6233    320.810     -1.252      0.211   -1030.404     227.158
start station id_174   -279.5655    331.586     -0.843      0.399    -929.466     370.335
start station id_195    321.5278    341.021      0.943      0.346    -346.866     989.921
start station id_212   

$H_0$: Significant effect of age when predicting tripduration is 0. \
$H_A$: Significant effect of age when predicting tripduration is NOT 0.

Since the p-value is 0.000 (which is less than 0.05), we reject $ H_0$ and state $ H_A$. Significant effect of age when predicting tripduration is NOT 0.

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?

Age has a significant effect on predicting tripduration. Coefficient for age is 5.2368. This means that as age of rider increases, the tripduration increases. Thus Citi Bike can market towards the older generation people so that their tripduration is long.